## Data Import

In [1]:
import kagglehub
import pandas as pd
import numpy as np
# Download latest version
path = kagglehub.dataset_download("devondev/financial-anomaly-data")

print("Path to dataset files:", path)
df=pd.read_csv(path+'/financial_anomaly_data.csv')
df.dropna(inplace= True)

100%|██████████| 2.92M/2.92M [00:01<00:00, 2.50MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/devondev/financial-anomaly-data/versions/1


## Feature Engineering

In [2]:
# Feature: TransactionCount
transaction_count = df.groupby('AccountID')['TransactionID'].count()
df['TransactionCount'] = df['AccountID'].map(transaction_count)

# Feature: MeanAmount
mean_amount = df.groupby('AccountID')['Amount'].mean()
df['MeanAmount'] = df['AccountID'].map(mean_amount)

# Feature: StdAmount
std_amount = df.groupby('AccountID')['Amount'].std().fillna(0)
df['StdAmount'] = df['AccountID'].map(std_amount)

# Feature: UniqueMerchants
unique_merchants = df.groupby('AccountID')['Merchant'].nunique()
df['UniqueMerchants'] = df['AccountID'].map(unique_merchants)

# Feature: ActiveDays
df['TransactionDate'] =pd.to_datetime(df['Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')
active_days = df.groupby('AccountID')['TransactionDate'].nunique()
df['ActiveDays'] = df['AccountID'].map(active_days)

print(df.head())


          Timestamp TransactionID AccountID    Amount   Merchant  \
0  01-01-2023 08:00       TXN1127      ACC4  95071.92  MerchantH   
1  01-01-2023 08:01       TXN1639     ACC10  15607.89  MerchantH   
2  01-01-2023 08:02        TXN872      ACC8  65092.34  MerchantE   
3  01-01-2023 08:03       TXN1438      ACC6     87.87  MerchantE   
4  01-01-2023 08:04       TXN1338      ACC6    716.56  MerchantI   

  TransactionType     Location  TransactionCount    MeanAmount     StdAmount  \
0        Purchase        Tokyo             14456  49623.778060  28875.132690   
1        Purchase       London             14362  49729.927676  28797.732137   
2      Withdrawal       London             14402  50481.295047  29290.474359   
3        Purchase       London             14352  50038.083433  28804.031520   
4        Purchase  Los Angeles             14352  50038.083433  28804.031520   

   UniqueMerchants     TransactionDate  ActiveDays  
0               10 2023-01-01 08:00:00       14456  
1   

## Anomaly Dataset creation

In [3]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Drop irrelevant columns
features = df.drop(['TransactionID', 'Timestamp', 'TransactionDate'], axis=1)

# Label encode categorical columns
label_encoder = LabelEncoder()
features['Merchant'] = label_encoder.fit_transform(features['Merchant'])
features['TransactionType'] = label_encoder.fit_transform(features['TransactionType'])
features['AccountID'] = label_encoder.fit_transform(features['AccountID'])
features['Location'] = label_encoder.fit_transform(features['Location'])

# Scale numerical features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

scaled_df = pd.DataFrame(scaled_features, columns=features.columns)
print(scaled_df.head())


   AccountID    Amount  Merchant  TransactionType  Location  TransactionCount  \
0   0.461173  1.545884  0.870183        -1.225041  1.414357         -0.080069   
1  -1.391974 -1.185041  0.870183        -1.225041 -1.415492         -0.931569   
2   1.387746  0.515582 -0.174726         1.228467 -1.415492         -0.569229   
3   0.924459 -1.718415 -0.174726        -1.225041 -1.415492         -1.022154   
4   0.924459 -1.696809  1.218485        -1.225041 -0.708030         -1.022154   

   MeanAmount  StdAmount  UniqueMerchants  ActiveDays  
0   -1.660833  -0.568613              0.0   -0.080069  
1   -1.282714  -0.768854              0.0   -0.931569  
2    1.393754   0.505910              0.0   -0.569229  
3   -0.185023  -0.752557              0.0   -1.022154  
4   -0.185023  -0.752557              0.0   -1.022154  


In [4]:
from sklearn.ensemble import IsolationForest

# Train Isolation Forest
iso_forest = IsolationForest(contamination=0.01, random_state=42)
iso_labels = iso_forest.fit_predict(scaled_df)

df['IsoForest_Anomaly'] = iso_labels
print(df['IsoForest_Anomaly'].value_counts())


IsoForest_Anomaly
 1    214795
-1      2165
Name: count, dtype: int64


In [5]:
from sklearn.cluster import DBSCAN

# Train DBSCAN
dbscan = DBSCAN(eps=2, min_samples=5)
dbscan_labels = dbscan.fit_predict(scaled_features)

df['DBSCAN_Anomaly'] = dbscan_labels
print(df['DBSCAN_Anomaly'].value_counts())


DBSCAN_Anomaly
 0    86923
 4    57662
 2    28636
 3    14701
 5    14628
 1    14400
-1       10
Name: count, dtype: int64


In [6]:
df['Anomaly'] = (df['IsoForest_Anomaly'] == -1) | (df['DBSCAN_Anomaly'] == -1)
anomalies = df[df['Anomaly']]
print(f"Total anomalies detected: {len(anomalies)}")

print(anomalies.head())


Total anomalies detected: 2175
            Timestamp TransactionID AccountID    Amount   Merchant  \
56   01-01-2023 08:56       TXN1070      ACC5  88350.57  MerchantI   
154  01-01-2023 10:34       TXN1313      ACC5  10187.23  MerchantF   
179  01-01-2023 10:59       TXN1546      ACC5  87677.80  MerchantA   
425  01-01-2023 15:05        TXN962     ACC15  20576.67  MerchantA   
427  01-01-2023 15:07       TXN1802      ACC5  12532.10  MerchantH   

    TransactionType Location  TransactionCount    MeanAmount     StdAmount  \
56         Purchase    Tokyo             14630  50194.706156  30071.156125   
154      Withdrawal   London             14630  50194.706156  30071.156125   
179      Withdrawal    Tokyo             14630  50194.706156  30071.156125   
425      Withdrawal   London             14701  50371.894606  28860.449456   
427      Withdrawal    Tokyo             14630  50194.706156  30071.156125   

     UniqueMerchants     TransactionDate  ActiveDays  IsoForest_Anomaly  \
56  

In [14]:
anomalies.shape

(2175, 16)

In [7]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 29.6 MB/s eta 0:00:00


In [8]:
import mysql.connector
from sqlalchemy import create_engine

host = 'sql12.freesqldatabase.com'
port = 3306
database = 'sql12759851'
user = 'sql12759851'
password = 'QDv4j2W7kE'

try:
    conn = mysql.connector.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    print("Successfully connected to the database!")
    cursor = conn.cursor()
    print("Cursor created successfully!")

except mysql.connector.Error as err:
    print(f"Error: {err}")



Successfully connected to the database!
Cursor created successfully!


In [10]:
from sqlalchemy import create_engine

engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}')


In [11]:
# Save anomalies DataFrame to MySQL
table_name = 'anomalies_detected'  # Table name

try:
    anomalies.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
    print(f"DataFrame saved successfully to table '{table_name}'.")
except Exception as e:
    print("Error saving DataFrame:", e)


DataFrame saved successfully to table 'anomalies_detected'.


In [13]:
# Verify saved data
query = f"SELECT COUNT(Timestamp) FROM {table_name} ;"  # Retrieve first 5 rows
cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)


(2175,)
